### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "German"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = German
Lang Id = 8
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4,https://www.youtube.com/watch?v=E5wrYo4PLM4&t=...
1,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k,https://www.youtube.com/watch?v=m9hZzWA0R3k&t=...
2,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY,https://www.youtube.com/watch?v=XGhAlBQ3gOY&t=...
3,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs,https://www.youtube.com/watch?v=X0g4THI7vEs&t=...
4,aber ist es nicht so dass du,353,354,aber ist es nicht so dass du eigentlich,X10OIhdciIw,https://www.youtube.com/watch?v=X10OIhdciIw&t=...
...,...,...,...,...,...,...
195,den ich nicht,71,72,den ich nicht mag,Yezhi_a8N2Y,https://www.youtube.com/watch?v=Yezhi_a8N2Y&t=71s
196,und ich so ja,679,681,und ich so ja okay,nuLExDdcYHQ,https://www.youtube.com/watch?v=nuLExDdcYHQ&t=...
197,und nicht sie,234,237,und nicht sie,doAGXI0X9zU,https://www.youtube.com/watch?v=doAGXI0X9zU&t=...
198,dass eine der,1959,1961,nur dass eine der,vDFwKGaCrFs,https://www.youtube.com/watch?v=vDFwKGaCrFs&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,oder da würde er sagen da ich glaube es geht l...,1499,1505,oder da würde er sagen da ich glaube es geht l...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
1,du denn jetzt machen oder sagen ja was können ...,2046,2051,du denn jetzt machen oder sagen ja was können ...,L22SNtHYLCw,https://www.youtube.com/watch?v=L22SNtHYLCw&t=...
2,also wenn dann jetzt ja wenn dann wollen wir j...,457,462,also wenn dann jetzt ja wenn dann wollen wir j...,dKN8OpO1Zpw,https://www.youtube.com/watch?v=dKN8OpO1Zpw&t=...
3,wenn mir ganz klar werden würde es gibt keinen...,580,585,wenn mir ganz klar werden würde es gibt keinen...,3Ie7_qSn0nE,https://www.youtube.com/watch?v=3Ie7_qSn0nE&t=...
4,auch wenn ich sie nicht sehen was wird dann di...,84,91,auch wenn ich sie nicht sehen was wird dann di...,J8a_qUpkpq4,https://www.youtube.com/watch?v=J8a_qUpkpq4&t=84s
...,...,...,...,...,...,...
195,werden klar jetzt hast du vielleicht sagen hey...,240,243,optimiert werden klar jetzt hast du vielleicht...,4nreqmvk3I8,https://www.youtube.com/watch?v=4nreqmvk3I8&t=...
196,und wir waren uns aber sicher damit wir haben ...,518,522,und wir waren uns aber sicher damit wir haben ...,AHeo9wfGrl0,https://www.youtube.com/watch?v=AHeo9wfGrl0&t=...
197,wir ein und dann haben wir einen weg vor uns u...,147,153,da steigen wir ein und dann haben wir einen we...,qTIVI3VB15g,https://www.youtube.com/watch?v=qTIVI3VB15g&t=...
198,ja wirklich kein nichts mehr was man sich einf...,1527,1531,ja wirklich kein nichts mehr was man sich einf...,OLLPmgdUqPc,https://www.youtube.com/watch?v=OLLPmgdUqPc&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ich glaube sie wollte sagen hey du bist nur ei...,3278,3285,ich glaube sie wollte sagen hey du bist nur ei...,twfbyHqyITc,https://www.youtube.com/watch?v=twfbyHqyITc&t=...
1,das hat damit zu tun dass wir einfach wissen d...,2804,2809,das hat damit zu tun dass wir einfach wissen d...,V1rKsO0okzY,https://www.youtube.com/watch?v=V1rKsO0okzY&t=...
2,die man dann auch wieder sagen musste hey lass...,171,177,die man dann auch wieder sagen musste hey lass...,0Cj9zh90gpY,https://www.youtube.com/watch?v=0Cj9zh90gpY&t=...
3,nehmen dann sollte es an einer stelle sein an ...,369,376,und wenn wir einen sticker nehmen dann sollte ...,VkcA1BcxD6s,https://www.youtube.com/watch?v=VkcA1BcxD6s&t=...
4,wenn nicht sogar schon vorher haben uns halt i...,135,140,wenn nicht sogar schon vorher haben uns halt i...,51AJlbQjqMg,https://www.youtube.com/watch?v=51AJlbQjqMg&t=...
...,...,...,...,...,...,...
195,eigentlich also wo man ja eigentlich sehr star...,655,660,eigentlich also wo man ja eigentlich sehr star...,ousfwgt35QE,https://www.youtube.com/watch?v=ousfwgt35QE&t=...
196,wer weiß schon was uns zwei der lange weg des ...,98,103,wer weiß schon was uns zwei der lange weg des ...,ZSdgrofAyE0,https://www.youtube.com/watch?v=ZSdgrofAyE0&t=98s
197,du super weiter so vielen lieben dank gut fast...,1078,1085,du super weiter so vielen lieben dank gut fast...,GBervqjuUbA,https://www.youtube.com/watch?v=GBervqjuUbA&t=...
198,wir sind wir sind doch alle großen wissen noch...,162,167,wir sind wir sind doch alle großen wissen noch...,Gn1tRduX2HA,https://www.youtube.com/watch?v=Gn1tRduX2HA&t=...


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,lassen wir haben nachher sehr oft da drüber ge...,608,618,ja meine frau hat sich das nie so anmerken las...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
1,einige haben ja ganz schön viel erlebt ich den...,366,381,einige haben ja ganz schön viel erlebt ich den...,_W671aWBZL4,https://www.youtube.com/watch?v=_W671aWBZL4&t=...
2,eine gewisse sicherheit bekommen dass das was ...,135,147,eine gewisse sicherheit bekommen dass das was ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
3,in diese welt gut dann können wir uns vielleic...,1552,1560,in diese welt gut dann können wir uns vielleic...,BS0krBY3O8I,https://www.youtube.com/watch?v=BS0krBY3O8I&t=...
4,und normalerweise bricht meine haut ein paar t...,447,458,ich habe gerade meine periode und normalerweis...,niPfa1UCmaE,https://www.youtube.com/watch?v=niPfa1UCmaE&t=...
...,...,...,...,...,...,...
195,irgendwas was nichts mit den hausaufgaben zu t...,280,286,irgendwas was nichts mit den hausaufgaben zu t...,MoAPdaWyzz4,https://www.youtube.com/watch?v=MoAPdaWyzz4&t=...
196,in der heutigen situation heißt es nicht nur d...,60,66,in der heutigen situation heißt es nicht nur d...,JwyJIHjNM5s,https://www.youtube.com/watch?v=JwyJIHjNM5s&t=60s
197,die butter muss natürlich weich sein klar die ...,118,124,die butter muss natürlich weich sein klar die ...,7NWdGGzURdQ,https://www.youtube.com/watch?v=7NWdGGzURdQ&t=...
198,trinken sie zum beispiel jedes mal direkt nach...,46,53,trinken sie zum beispiel jedes mal direkt nach...,7Mrk10mj0hY,https://www.youtube.com/watch?v=7Mrk10mj0hY&t=46s


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,die schon lange nicht mehr benutzt wurde und i...,55,65,wie so eine alte kartoffel die schon lange nic...,5b46anicJdI,https://www.youtube.com/watch?v=5b46anicJdI&t=55s
1,es ist auch eine große erleichterung dass wir ...,727,737,es ist auch eine große erleichterung dass wir ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
2,ja die frage wird oft gestellt man macht sich ...,1272,1283,ja die frage wird oft gestellt man macht sich ...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
3,emotionale momente wenn so ein brief bei einer...,5,18,emotionale momente wenn so ein brief bei einer...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=5s
4,eine gewisse sicherheit bekommen dass das was ...,135,147,eine gewisse sicherheit bekommen dass das was ...,gweZAS7SiOU,https://www.youtube.com/watch?v=gweZAS7SiOU&t=...
...,...,...,...,...,...,...
195,heisst nicht unbedingt weit weg zu reisen sond...,376,383,zwischenjahr heisst nicht unbedingt weit weg z...,HLwt7HpweWo,https://www.youtube.com/watch?v=HLwt7HpweWo&t=...
196,der da irgendwie acht stunden für nicht gerade...,228,234,der da irgendwie acht stunden für nicht gerade...,4vH0XDy4Czg,https://www.youtube.com/watch?v=4vH0XDy4Czg&t=...
197,wobei der prozess dann für mich nicht mehr so ...,1662,1678,wobei der prozess dann für mich nicht mehr so ...,HI7vQ2yNqBc,https://www.youtube.com/watch?v=HI7vQ2yNqBc&t=...
198,wollte er erst gar nichts von einem vertrag wi...,245,252,wollte er erst gar nichts von einem vertrag wi...,0Cj9zh90gpY,https://www.youtube.com/watch?v=0Cj9zh90gpY&t=...


In [14]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,er zu dem zeitpunkt noch ein patient ist dass ...,954,966,er zu dem zeitpunkt noch ein patient ist dass ...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=...
1,ja wir freuen uns total mit ihnen dass das gek...,1144,1154,ja wir freuen uns total mit ihnen dass das gek...,rHKdeRoDJDs,https://www.youtube.com/watch?v=rHKdeRoDJDs&t=...
2,dürfen immer so meine ganz persönliche fragen ...,1724,1736,dürfen immer so meine ganz persönliche fragen ...,UwwRcluYoDc,https://www.youtube.com/watch?v=UwwRcluYoDc&t=...
3,emotionale momente wenn so ein brief bei einer...,1248,1262,emotionale momente wenn so ein brief bei einer...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=...
4,man eben keine transplantation anbieten kann o...,451,463,man eben keine transplantation anbieten kann o...,JRFCFvPe5no,https://www.youtube.com/watch?v=JRFCFvPe5no&t=...
...,...,...,...,...,...,...
195,früher in der schule war es ja noch einfach do...,153,159,früher in der schule war es ja noch einfach do...,IdOKNomMRSM,https://www.youtube.com/watch?v=IdOKNomMRSM&t=...
196,also können wir am nächsten tag selbst wenn wi...,213,221,also können wir am nächsten tag selbst wenn wi...,VJyLMDh0O64,https://www.youtube.com/watch?v=VJyLMDh0O64&t=...
197,aber ihr könnt auf jeden fall ab oder lassen u...,105,110,aber ihr könnt auf jeden fall ab oder lassen u...,zXC6n8C0EKM,https://www.youtube.com/watch?v=zXC6n8C0EKM&t=...
198,deswegen haben wir uns dagegen entschieden geh...,828,836,deswegen haben wir uns dagegen entschieden geh...,msWVQkbZIVI,https://www.youtube.com/watch?v=msWVQkbZIVI&t=...


In [15]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,doch noch bevor ich antworten konnte schrie si...,358,366,doch noch bevor ich antworten konnte schrie si...,9k2_UwkcYWo,https://www.youtube.com/watch?v=9k2_UwkcYWo&t=...
1,mal zurück zu ihrer tätigkeit als solche mich ...,758,774,mal zurück zu ihrer tätigkeit als solche mich ...,URZkhWJpxs4,https://www.youtube.com/watch?v=URZkhWJpxs4&t=...
2,ich muss sagen dafür dass mein schlafsack eige...,1177,1195,ich muss sagen dafür dass mein schlafsack eige...,XxqC2Ghsns4,https://www.youtube.com/watch?v=XxqC2Ghsns4&t=...
3,aber auch das ist nicht schlimm auch dort wird...,1425,1447,aber auch das ist nicht schlimm auch dort wird...,0m8UgZhUHT0,https://www.youtube.com/watch?v=0m8UgZhUHT0&t=...
4,er zu dem zeitpunkt noch ein patient ist dass ...,954,966,er zu dem zeitpunkt noch ein patient ist dass ...,prJrgq_RXKg,https://www.youtube.com/watch?v=prJrgq_RXKg&t=...
...,...,...,...,...,...,...
195,die tomaten bringen so ein bisschen frische mi...,300,306,die tomaten bringen so ein bisschen frische mi...,0xAVOsLqNPQ,https://www.youtube.com/watch?v=0xAVOsLqNPQ&t=...
196,manchmal auch als nachteil empfinde denn wenn ...,26,31,manchmal auch als nachteil empfinde denn wenn ...,glOUUqNz3gQ,https://www.youtube.com/watch?v=glOUUqNz3gQ&t=26s
197,ist für viele menschen geeignet denke ich sich...,141,148,ist für viele menschen geeignet denke ich sich...,iBfcPxJS8hk,https://www.youtube.com/watch?v=iBfcPxJS8hk&t=...
198,natürlich muss man sich ein bisschen umstellen...,70,78,natürlich muss man sich ein bisschen umstellen...,AmNlRB_oqPU,https://www.youtube.com/watch?v=AmNlRB_oqPU&t=70s


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1671722344
  nanos: 945807000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass